In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
tokenizer=Tokenizer()

In [ ]:
tokenizer.fit_on_texts([text])

In [ ]:
vocab_size=len(tokenizer.word_index)+1
print('단어 집합의 크기 : %d' % vocab_size)

In [ ]:
tokenizer.index_word

In [ ]:
text.split('\n')

In [ ]:
tokenizer.texts_to_sequences(['경마장에 있는 말이 뛰고 있다'])

In [ ]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)): # [2,3,1,4,5]
        sequence = encoded[:i+1] # [2,3] , [2,3,1] ..
        sequences.append(sequence) # [[2,3],[2,3,1]....]

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

2 -> model -> 3
2,3 -> model -> 1  가장 마지막에 들어간 것이 타겟이라고 생각 ( y : target data)
입력길이를 얼마나 할지 고민해야된다. (1 ~ 5까지)

In [ ]:
sequences

In [ ]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [ ]:
sequences

In [ ]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
X

In [ ]:
y = to_categorical(y, num_classes=vocab_size)


In [ ]:
y

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

In [ ]:
embedding_dim = 10 # 임베딩 차원 : 10차원
hidden_units = 32
# 임베딩? 단엉를 벡터공간 표현 -> 임베딩 벡터 공간 : 11차원(단어 의 종류 개수)+1 = 12차원
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # 12차원의 데이터를 10차원 공간 데이터로 표혀해라!

model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    current_word = '경마장에 있는'
    init_word = current_word
    sentence = ''
    encoded = tokenizer.text_to_sequences([current_word])[0]
    print(encoded)# [2,3]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩


        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ArticlesApril2018.csv')
df.head
#head 라인 열에있는 자료들을 추출한다.

In [ ]:
print('열의 개수: ',len(df.columns))
print(df.columns)

In [ ]:
print(df['headline'].isnull().values.any())

In [ ]:
headline=[]
headline.extend(list(df.headline.values))

In [ ]:
headline

In [ ]:
def repreprocessing(raw_sentence): #
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

In [ ]:
punctuation #

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1 # 3495개
print('단어 집합의 크기 : %d' % vocab_size)# 단어.. 집합의 전체 크기

In [ ]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

In [ ]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

In [ ]:
index_to_word

In [ ]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [ ]:
sequences

In [ ]:
y

In [ ]:
sequences= np.array(sequences)
X= sequences[:,:-1] #
y= sequences[:,-1]

In [ ]:
vocab_size # 차원이 3495 차원이다!

# ex ) 1025 -> 0000......0010000000000.... 00

In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y

In [ ]:
y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # 3494 -> 10차원으로
model.add(LSTM(hidden_units))# lstm 셀 출력 : 128차원
model.add(Dense(vocab_size, activation='softmax')) # Dense Fully connected 레이어! ,
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

In [ ]:
print(sentence_generation(model, tokenizer, 'i', 10))